In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

from tqdm import tqdm

[nltk_data] Downloading package punkt to /home/noble6/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd drive/MyDrive/datasets

/content/drive/MyDrive/datasets


## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена: 
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ. 
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size] 
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [26]:
class NamesDataset(Dataset):
    def __init__(self, file_path):
        self.special_tokens = {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3}
        self.word2index = {}
        self.index2word = {}
        self.max_len = 0
        self.data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                self.max_len = max(self.max_len, len(word))
                self.data.append(word)
                for char in word:
                    if char not in self.word2index:
                        index = len(self.word2index)
                        self.word2index[char] = index
                        self.index2word[index] = char
        for token in self.special_tokens:
            index = len(self.word2index)
            self.word2index[token] = index
            self.index2word[index] = token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        word = self.data[index]
        input_seq = [self.word2index['<SOS>']] + [self.word2index.get(char, self.word2index['<UNK>']) for char in word] + [self.word2index['<EOS>']]
        input_seq += [self.word2index['<PAD>']] * (self.max_len - len(input_seq))
        target_seq = input_seq[1:] + [self.word2index['<PAD>']]
        return torch.tensor(input_seq), torch.tensor(target_seq)

In [27]:
dataset = NamesDataset('data/name_rus.txt')
print(len(dataset))
input_seq, target_seq = dataset[12]

input_seq, target_seq

1988


(tensor([32,  0, 18, 19,  7, 21,  7,  9,  4,  0, 33, 30, 30]),
 tensor([ 0, 18, 19,  7, 21,  7,  9,  4,  0, 33, 30, 30, 30]))

In [69]:
dataset[18][1].shape

torch.Size([13])

In [23]:
dataset.word2index["<EOS>"]

33

In [28]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
# for input_seq, target_seq in dataloader:
    # print(input_seq.shape, target_seq.shape)
    # break

In [29]:
next(iter(dataloader))

[tensor([[32,  0,  1,  2,  3,  4,  5,  6, 33, 30, 30, 30, 30]]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6, 33, 30, 30, 30, 30, 30]])]

In [38]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.rnn = nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=2)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        emb = self.embedding(input_seq)
        output, _ = self.rnn(emb)
        output = output.reshape(-1, output.shape[2])
        output = self.linear(output)
        return output

def generate_name(model, dataset, max_len=20):
    with torch.no_grad():
        input_seq = torch.tensor([[dataset.word2index['<SOS>']]])
        # h_t = None
        name = ''
        while True:
            output = model(input_seq.cuda())
            probs = nn.functional.softmax(output[-1], dim=0)
            next_token = torch.multinomial(probs, num_samples=1).item()
            if next_token == dataset.word2index["<EOS>"] or len(name) >= max_len:
                break
            name += dataset.index2word[next_token]
            input_seq = torch.tensor([[next_token]])
            # h_t = output[-1].unsqueeze(0)
        return name

def train(model, dataloader, dataset, num_epochs, lr, device):
    # dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.word2index['<PAD>'])
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        total_loss = 0
        for input_seq, target_seq in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            # print(123)
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)
            optimizer.zero_grad()
            output = model(input_seq)
            loss = criterion(output, target_seq.reshape(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
        if (epoch+1) % 5 == 0:
            names = []
            for i in range(2):
                name = generate_name(model, dataset)
                names.append(name)
            print(f"Предсказанные имена: {names}")

In [31]:
model = RNNModel(len(dataset.word2index), 16, 32)
train(model, dataloader, dataset, 50, 0.01, 'cuda')

Epoch 1/50: 100%|██████████| 1988/1988 [00:03<00:00, 543.83it/s]


Epoch 1/50, Loss: 2.0393


Epoch 2/50: 100%|██████████| 1988/1988 [00:03<00:00, 601.13it/s]


Epoch 2/50, Loss: 2.0733


Epoch 3/50: 100%|██████████| 1988/1988 [00:03<00:00, 531.46it/s]


...

Epoch 48/50, Loss: 2.0456


Epoch 49/50: 100%|██████████| 1988/1988 [00:03<00:00, 544.88it/s]


Epoch 49/50, Loss: 2.0557


Epoch 50/50: 100%|██████████| 1988/1988 [00:03<00:00, 569.36it/s]

Epoch 50/50, Loss: 2.0538
Предсказанные имена: ['юранюранич', 'юрадоминярьканилюряр']


In [32]:
name = generate_name(model, dataset)


In [33]:
name

'юриадомич'

In [ ]:
import pickle

In [34]:
# Сохранение модели в файл
with open("rnn_name_generator.pickle", "wb") as f:
    pickle.dump(model, f)

In [35]:
# Загрузка модели из файла
with open("rnn_name_generator.pickle", "rb") as f:
    model = pickle.load(f)

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет. 

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова) 
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


In [2]:
import string

with open('data/Война и мир. Том 1.Лев Толстой.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Remove newlines and extra spaces
text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ')
text = ' '.join(text.split())

seq_len = 100
step = 5

sentences = []
next_chars = []
for i in range(0, len(text) - seq_len, step):
    sentences.append(text[i:i+seq_len])
    next_chars.append(text[i+seq_len])

print('Number of sequences:', len(sentences))

Number of sequences: 142798


In [3]:
# Create char to index and index to char dictionaries
chars = sorted(list(set(text)))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

In [4]:
import torch
import torch.nn as nn
import numpy as np

class TextGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.5):
        super(TextGenerator, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, h):
        embedded = self.embedding(x)
        out, h = self.lstm(embedded, h)
        out = self.fc(out[:, -1, :])
        return out, h
    
    def init_hidden(self, batch_size, device):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device))

input_size = len(string.printable)
hidden_size = 256
output_size = len(string.printable)

model = TextGenerator(input_size, hidden_size, output_size, num_layers=2, dropout=0.5)
model.to("cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
def train(model, optimizer, criterion, sentences, next_chars, batch_size=128, num_epochs=10, device="cuda"):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        hidden = model.init_hidden(batch_size)
        
        for i in range(0, len(sentences) - batch_size, batch_size):
            inputs = np.zeros((batch_size, seq_len))
            targets = np.zeros((batch_size,))
            
            for j in range(batch_size):
                inputs[j] = [char_to_index[c] for c in sentences[i+j]]
                targets[j] = char_to_index[next_chars[i+j]]
                
            inputs = torch.LongTensor(inputs).to(device)
            targets = torch.LongTensor(targets).to(device)
            
            hidden = tuple([h.detach() for h in hidden])
            
            optimizer.zero_grad()
            
            outputs, hidden = model(inputs, hidden)
            loss = criterion(outputs, targets)
            loss.backward()
            
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            
            optimizer.step()
            
            total_loss += loss.item()
            
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, total_loss))

In [6]:
index_to_char

{0: ' ',
 1: '!',
 2: "'",
 3: '(',
 4: ')',
 5: '*',
 6: ',',
 7: '-',
 8: '.',
 9: '0',
 10: '1',
 11: '2',
 12: '3',
 13: '4',
 14: '5',
 15: '6',
 16: '7',
 17: '8',
 18: '9',
 19: ':',
 20: ';',
 21: '?',
 22: 'A',
 23: 'B',
 24: 'C',
 25: 'D',
 26: 'E',
 27: 'F',
 28: 'G',
 29: 'H',
 30: 'I',
 31: 'J',
 32: 'K',
 33: 'L',
 34: 'M',
 35: 'N',
 36: 'O',
 37: 'P',
 38: 'Q',
 39: 'R',
 40: 'S',
 41: 'T',
 42: 'U',
 43: 'V',
 44: 'W',
 45: 'X',
 46: 'Z',
 47: '[',
 48: ']',
 49: '`',
 50: 'a',
 51: 'b',
 52: 'c',
 53: 'd',
 54: 'e',
 55: 'f',
 56: 'g',
 57: 'h',
 58: 'i',
 59: 'j',
 60: 'k',
 61: 'l',
 62: 'm',
 63: 'n',
 64: 'o',
 65: 'p',
 66: 'q',
 67: 'r',
 68: 's',
 69: 't',
 70: 'u',
 71: 'v',
 72: 'w',
 73: 'x',
 74: 'y',
 75: 'z',
 76: '«',
 77: '»',
 78: 'А',
 79: 'Б',
 80: 'В',
 81: 'Г',
 82: 'Д',
 83: 'Е',
 84: 'Ж',
 85: 'З',
 86: 'И',
 87: 'Й',
 88: 'К',
 89: 'Л',
 90: 'М',
 91: 'Н',
 92: 'О',
 93: 'П',
 94: 'Р',
 95: 'С',
 96: 'Т',
 97: 'У',
 98: 'Ф',
 99: 'Х',
 100: 'Ц',

In [7]:
def generate_text(model, prime, length, device="cpu"):
    model.to(device)
    model.eval()
    hidden = model.init_hidden(1, device=device)
    
    prime = [char_to_index[c] for c in prime]
    prime = torch.LongTensor(prime).unsqueeze(0).to(device)
    
    _, hidden = model(prime, hidden)
    
    generated = prime
    
    for i in range(length):
        outputs, hidden = model(generated[:, -seq_len:], hidden)
        _, topi = outputs.topk(1)
        generated = torch.cat((generated, topi), dim=1)
        
    generated = generated.squeeze().tolist()
    text = ''.join([index_to_char[i] for i in generated])
    
    return text

In [8]:
class TextDataset(Dataset):
    def __init__(self, text, seq_len, step):
        self.seq_len = seq_len
        self.step = step
        
        self.sentences = []
        self.next_chars = []
        for i in range(0, len(text) - seq_len, step):
            self.sentences.append(text[i:i+seq_len])
            self.next_chars.append(text[i+seq_len])
            
        self.char_to_index = {c: i for i, c in enumerate(sorted(list(set(text))))}
        
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        x = [self.char_to_index[c] for c in self.sentences[idx]]
        y = self.char_to_index[self.next_chars[idx]]
        return torch.LongTensor(x), torch.LongTensor([y])
    
    @property
    def vocab_size(self):
        return len(self.char_to_index)

In [9]:
dataset = TextDataset(text, seq_len=30, step=5)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [11]:
def train(model, dataloader, dataset, num_epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    
    for epoch in range(num_epochs):
        total_loss = 0
        model.train()    
        for x, y in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            
            hidden = model.init_hidden(x.size(0), device)
            output, hidden = model(x, hidden)
            
            loss = criterion(output.view(-1, dataset.vocab_size), y.view(-1))
            loss.backward()
            
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            
            optimizer.step()
            
            total_loss += loss.item()
            
        avg_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
        
        if (epoch+1) % 10 == 0:
            model.eval()
            generated_text = generate_text(model, 'Война', 100, device=device)
            print(f"Generated text:\n{generated_text}")

In [12]:
dataset.vocab_size

146

In [13]:
model = TextGenerator(input_size=dataset.vocab_size, hidden_size=64, output_size=dataset.vocab_size, num_layers=2, dropout=0.5)

train(model, dataloader, dataset, num_epochs=10, lr=0.003, device="cuda")

Epoch 1/10: 100%|██████████| 1116/1116 [00:07<00:00, 154.27it/s]


Epoch 1/10, Loss: 2.7449


Epoch 2/10: 100%|██████████| 1116/1116 [00:06<00:00, 161.05it/s]


Epoch 2/10, Loss: 2.3977


Epoch 3/10: 100%|██████████| 1116/1116 [00:06<00:00, 160.04it/s]


...

Epoch 8/10, Loss: 2.0887


Epoch 9/10: 100%|██████████| 1116/1116 [00:07<00:00, 159.13it/s]


Epoch 9/10, Loss: 2.0720


Epoch 10/10: 100%|██████████| 1116/1116 [00:07<00:00, 152.52it/s]


Epoch 10/10, Loss: 2.0540
Generated text:
Война с все постарал с в то в то все постарал с в то в то все постарал с в то в то все постарал с в то в 


In [14]:
train(model, dataloader, dataset, num_epochs=40, lr=0.003, device="cuda")

Epoch 1/40: 100%|██████████| 1116/1116 [00:07<00:00, 155.25it/s]


Epoch 1/40, Loss: 2.0447


Epoch 2/40: 100%|██████████| 1116/1116 [00:06<00:00, 160.83it/s]


Epoch 2/40, Loss: 2.0313


Epoch 3/40: 100%|██████████| 1116/1116 [00:06<00:00, 161.86it/s]


...

Epoch 21/40, Loss: 1.9212


Epoch 22/40: 100%|██████████| 1116/1116 [00:07<00:00, 154.33it/s]


Epoch 22/40, Loss: 1.9194


Epoch 23/40:  78%|███████▊  | 874/1116 [00:05<00:01, 157.61it/s]


KeyboardInterrupt: 

In [15]:
prime = 'Толстой вышел на улицу'
length = 100

generated_text = generate_text(model, prime, length)
print(generated_text)

Толстой вышел на улицу и приставил с приставил с приставил с приставил с приставил с приставил с приставил с приставил с п
